# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import os

os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface'
os.environ['HF_DATASETS_CACHE'] = '/root/autodl-tmp/huggingface/datasets'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/huggingface/hub'

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，代表阳、强、刚、动等含义。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象，以及父亲、君主、领导等人类社会中的阳刚之气。乾卦具有刚强、积极、主动、进取等特性，是八卦中最为积极向上、富有生命力的卦象之一。

在易经中，乾卦的卦辞为：“天行健，君子以自强不息。”这意味着乾卦鼓励人们要效仿天的行健，不断自强，不断前进，以达到最终的成功。乾卦还象征着红日、狮子、君主等，这些事物都是充满活力、具有领导力的。因此，乾卦在卜辞中常被用来指导人们如何在人生道路上坚持不懈、积极进取。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它是由两个阳爻夹一个阴爻构成。讼卦象征着诉讼、争端、诉讼等，意味着人们在处理问题和冲突时可能会遇到困难和阻碍。

在易经中，讼卦的卦辞为：“天雷震， lo Lo（隆） lo（洛），君子以听。”这意味着在处理争端和诉讼时，君子应该保持冷静和理智，倾听双方的意见，并做出公正的判断。

讼卦的建议是：“天雷震，示有所震动。君子以听，有所不合。”这意味着在处理争端和诉讼时，君子应该倾听双方的意见，并做出公正的判断，而不是盲目地支持某一方。

总的来说，讼卦提醒人们在处理问题和冲突时要保持冷静和理智，倾听双方的意见，并做出公正的判断。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240313_173956"#"20240225_222843"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦中的一卦，象征天。它由六个阳爻组成，代表着纯粹的阳刚之气，意味着兴盛和强健。乾卦所代表的天行健，鼓励人们以天道为法，自强不息。其含义是通泰、吉利、强健，但需要隐藏收敛，不可过于急躁。在运势上，乾卦代表着事顺意顺、如意，但需要保持冷静，不可过于享受。在事业、经商、求名、婚恋和决策方面，都需要保持稳健和正面应对，不可过于激进。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦中的第一卦，象征着天。它由六个阳爻组成，代表着纯粹的阳刚之气，意味着兴盛和强健。乾卦所代表的天行健，鼓励人们以天道为法，自强不息。其含义是大吉大利，表示顺利通达，但必须行正道才能永远亨通。乾卦的核心哲学是坚强、正直、克己奉公，需要谨防过于强大。在事业上，乾卦象征着一切顺利、如意，但也需要时刻警惕，保持谦逊和冷静；在经商上，表示大好的发展机会，但需要冷静理性对待，不可急躁；在婚恋方面，阳刚之气盛而阴柔之气衰，需要均衡相济才能取得美满的结果。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦是一个充满警惕和挑战的卦象。它由乾卦（天）与坎卦（水）相叠而成，代表着天和水背道而驰，形成争讼的局面。虽然中间可能有利可图，却极易陷入终凶的境地。讼卦提醒人们，在谋事之初必须审慎小心，杜绝争讼为要，避免陷入是非之争。', 'content': '\n讼卦隐含的核心哲学是：即使事情起初看似顺利，但随后可能陷入凶险。因此，不应固执已见，而是应该在适当时候退让解争。这样做，可以避免意外之灾，安于正理。同时，讼卦也提醒着人们，不要贪图不义之财，应坚持公正、公平、互利的原则，这样才能化解冲突，取得良好的结果。在决策上，应当遵循现实情况，知足适可而止，避免好胜而得罪他人，带来诉讼之灾。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个充满警惕和挑战的卦象。它由乾卦（天）与坎卦（水）相叠而成，代表着天和水背道而驰，形成争讼的局面。虽然中间可能有利可图，却极易陷入终凶的境地。讼卦提醒人们，在谋事之初必须审慎小心，杜绝争讼为要，避免陷入是非之争。

讼卦隐含的核心哲学是：即使事情起初看似顺利，但随后可能陷入凶险。因此，不应固执已见，而是应该在适当时候退让解争。这样做，可以避免意外之灾，安于正理。同时，讼卦也提醒着人们，不要贪图不义之财，应坚持公正、公平、互利的原则，这样才能化解冲突，取得良好的结果。在决策上，应当遵循现实情况，知足适可而止，避免好胜而得罪他人，带来诉讼之灾。


In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个充满智慧的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表着师，即军队，寓意着兵权，同时暗示着吉利。兵法云“兵贵神速”，因此，在军事策略上，迅速发兵并击退敌人是关键。

师卦的核心哲学是：兵者，奇变之首；神速为贵。在运势、事业、经商、求名、婚恋和决策等方面，都需要注意速度和灵活性。同时，也要注意避免快速行动带来不利影响。在好运时，要迅速抓住机会，迅速成长；在不顺利时，要耐心等待，避免因急躁而带来更坏后果。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

content:"师卦解释"
summary:"师卦代表总指挥的军情，象征着兵力和农力的结合。在《象辞》中，“地中有水”的卦象意味着君子应取法于大地容纳江河，收容和畜养大众。师卦的解释中，只有德高望重的长者统率军队才能吉祥无咎。这一卦象暗示着时运中包容别人，修行待时的时机，必须有财有库并珍惜，家宅方面有喜庆的联姻，健康方面需注意调气无忧。在传统解卦中，师卦意味着养兵聚众，出师攻伐的象征，需要正规行事与密切合作，忌独断独行和投机取巧。在事业、经商、求名和婚恋方面都需要谨慎小心，审时度势，严格要求自己，力求中正老成，方能取得成功。


In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的谦卦有什么警示？", base_model, qlora_model, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("既济卦和未济卦有什么差异？", base_model, qlora_model, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("乾卦和坤卦分别代表什么？", base_model, qlora_model, training_tag)

## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
# from peft import PeftModel, PeftConfig

# epochs = 3
# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

# config = PeftConfig.from_pretrained(peft_model_path)
# qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
# training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

ValueError: Can't find 'adapter_config.json' at 'models/THUDM/chatglm3-6b-epoch3'

In [ ]:
# base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
# base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
# base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [ ]:
# from peft import PeftModel, PeftConfig

# epochs = 50
# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

# config = PeftConfig.from_pretrained(peft_model_path)
# qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
# training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [ ]:
# base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
# base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
# base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)